## Common LLM applications

The goal of this section is to get your feet wet with several LLM applications and to show how easy it can be to get started with LLMs.

As you go through the examples, note the datasets, models, APIs, and options used.  These simple examples can be starting points when you need to build your own application.


In [1]:
# Libraries:
# [sacremoses](https://github.com/alvations/sacremoses) is for the translation model `Helsinki-NLP/opus-mt-en-es`

%pip install sacremoses==0.0.53

Note: you may need to restart the kernel to use updated packages.


In [2]:
%%time

from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import torch
import time

start_time = time.time()

device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    print(f"GPUs available: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("There is no GPU available. Using CPU")

GPUs available: 1
GPU 0: NVIDIA RTX A4000
CPU times: user 1 s, sys: 240 ms, total: 1.24 s
Wall time: 1.28 s


### Summarization

Summarization can take two forms:
* `extractive` (selecting representative excerpts from the text)
* `abstractive` (generating novel text summaries)

Here, we will use a model which does *abstractive* summarization.

**Background reading**: The [Hugging Face summarization task page](https://huggingface.co/docs/transformers/tasks/summarization) lists model architectures which support summarization. The [summarization course chapter](https://huggingface.co/course/chapter7/5) provides a detailed walkthrough.

In this section, we will use:
* **Data**: [xsum](https://huggingface.co/datasets/xsum) dataset, which provides a set of BBC articles and summaries.
* **Model**: [t5-small](https://huggingface.co/t5-small) model, which has 60 million parameters (242MB for PyTorch).  T5 is an encoder-decoder model created by Google which supports several tasks such as summarization, translation, Q&A, and text classification.

For more details, see the [Google blog post](https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html), [code on GitHub](https://github.com/google-research/text-to-text-transfer-transformer), or the [research paper](https://arxiv.org/pdf/1910.10683.pdf).


### This dataset provides 3 columns:
* `document`: the BBC article text
* `summary`: a "ground-truth" summary --> Note how subjective this "ground-truth" is.  Is this the same summary you would write?  This a great example of how many LLM applications do not have obvious "right" answers.
* `id`: article ID

In [3]:
xsum_dataset = load_dataset(
    'xsum', version='1.2.0', cache_dir='data'
)
print(xsum_dataset)

Found cached dataset xsum (/linux-data/llm-course/data/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})


In [4]:
xsum_sample = xsum_dataset['train'].select(range(10))
display(xsum_sample.to_pandas())

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984
5,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,20836172
7,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467
8,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845
9,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,30358490


#### We next use the Hugging Face `pipeline` tool to load a pre-trained model.  In this LLM pipeline constructor, we specify:
* `task`: This first argument specifies the primary task.  See [Hugging Face tasks](https://huggingface.co/tasks) for more information.
* `model`: This is the name of the pre-trained model from the [Hugging Face Hub](https://huggingface.co/models).
* `min_length`, `max_length`: We want our generated summaries to be between these two token lengths.
* `truncation`: Some input articles may be too long for the LLM to process.  Most LLMs have fixed limits on the length of input sequences.

This option tells the pipeline to truncate the input if needed.


In [5]:
summarizer = pipeline(
    task='summarization',
    model='t5-small',
    min_length=20,
    max_length=80,
    truncation=True,
    device=device,
    model_kwargs={'cache_dir':'data'},
)

In [6]:
# Apply to 1 article
summarizer(xsum_sample['document'][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [7]:
# Apply to a batch of articles
results = summarizer(xsum_sample['document'])

In [8]:
# Display the generated summary side-by-side with the reference summary and original document.
# We use Pandas to join the inputs and outputs together in a nice format.

display(
    pd.DataFrame.from_dict(results)
    .rename({'summary_text' : 'generated_summary'}, axis=1)
    .join(pd.DataFrame.from_dict(xsum_sample))[
        ['generated_summary', 'summary', 'document']
    ]
)

,generated_summary,summary,document
0,the full cost of damage in Newton Stewart is s...,Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one..."
1,a fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...
2,Sebastian Vettel will start third ahead of tea...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...
3,the 67-year-old is accused of committing the o...,A former Lincolnshire Police officer carried o...,"John Edward Bates, formerly of Spalding, Linco..."
4,a man receiving psychiatric treatment at the c...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...
5,Gregor Townsend gave a debut to powerhouse win...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...,Welsh cyclist Luke Rowe says changes to the sp...,Belgian cyclist Demoitie died after a collisio...
8,gundogan will not be fit for the start of the ...,Manchester City midfielder Ilkay Gundogan says...,"Gundogan, 26, told BBC Sport he ""can see the f..."
9,the crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened about 07:20 GMT at the junc...


### Few-shot learning

In few-shot learning tasks, you give the model an instruction, a few query-response examples of how to follow that instruction, and then a new query.  The model must generate the response for that new query.  This technique has pros and cons: it is very powerful and allows models to be reused for many more applications, but it can be finicky and require significant prompt engineering to get good and reliable results.

**Background reading**: See the [Wikipedia page on few-shot learning](https://en.wikipedia.org/wiki/Few-shot_learning_&#40;natural_language_processing&#41;) or [this Hugging Face blog about few-shot learning](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

In this section, we will use:
* **Task**: Few-shot learning can be applied to many tasks.  Here, we will do sentiment analysis, which was covered earlier.  However, you will see how few-shot learning allows us to specify custom labels, whereas the previous model was tuned for a specific set of labels.  We will also show other (toy) tasks at the end.  In terms of the Hugging Face `task` specified in the `pipeline` constructor, few-shot learning is handled as a `text-generation` task.
* **Data**: We use a few examples, including a tweet example from the blog post linked above.
* **Model**: [gpt-neo-1.3B](https://huggingface.co/EleutherAI/gpt-neo-1.3B), a version of the GPT-Neo model discussed in the blog linked above.  It is a transformer model with 1.3 billion parameters developed by Eleuther AI.  For more details, see the [code on GitHub](https://github.com/EleutherAI/gpt-neo) or the [research paper](https://arxiv.org/abs/2204.06745).

In [9]:
%%time

# We will limit the response length for our few-shot learning tasks.

few_shot_pipeline = pipeline (
    task='text-generation',
    model="EleutherAI/gpt-neo-1.3B",
    max_new_tokens=10,
    device=device,
    model_kwargs={'cache_dir': 'data'},
)

CPU times: user 7.07 s, sys: 1.39 s, total: 8.46 s
Wall time: 6.38 s


***Tip***: In the few-shot prompts below, we separate the examples with a special token "###" and use the same token to encourage the LLM to end its output after answering the query.  We will tell the pipeline to use that special token as the end-of-sequence (EOS) token below.

In [10]:
# Get the token ID for "###", which we will use as the EOS token below

eos_token_id = few_shot_pipeline.tokenizer.encode("###")[0]
print(eos_token_id)

21017


In [11]:
# Without any examples, the model output is inconsistent and usually incorrect

results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

    [Tweet]: "This new music video was incredible"
    [Sentiment]: """,
    eos_token_id=eos_token_id
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

    [Tweet]: "This new music video was incredible"
    [Sentiment]:   "Unbelievable"

The


In [12]:
# With one example. the model may or may not get the answer right

results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]: """,
    eos_token_id = eos_token_id
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

    [Tweet]: "This is the link to the article"
    [Sentiment]: Neutral
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]:           


In [13]:
# With 1 example for sentiment, the model is more likely to understand

results = few_shot_pipeline(
    """For each tweet, describe its sentiment:

    [Tweet]: "I hate it when my phone battery dies"
    [Sentiment]: "Negative"
    ###
    [Tweet]: "My day has been nice"
    [Sentiment]: "Positive"
    ###
    [Tweet]: "This is the link to the article"
    [Sentiment]: "Neutral"
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]: "Positive",
    ###
    [Tweet]: "This place sucks"
    [Sentiment]: "Negative",
    ###
    [Tweet]: "This new video clip was amazing"
    [Sentiment]: """,
    eos_token_id = eos_token_id
)

print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


For each tweet, describe its sentiment:

    [Tweet]: "I hate it when my phone battery dies"
    [Sentiment]: "Negative"
    ###
    [Tweet]: "My day has been nice"
    [Sentiment]: "Positive"
    ###
    [Tweet]: "This is the link to the article"
    [Sentiment]: "Neutral"
    ###
    [Tweet]: "This new music video was incredible"
    [Sentiment]: "Positive",
    ###
    [Tweet]: "This place sucks"
    [Sentiment]: "Negative",
    ###
    [Tweet]: "This new video clip was amazing"
    [Sentiment]:  "Neutral",
    ###



### Hugging Face APIs
 In this section, we dive into some more details on Hugging Face APIs.
* Search and sampling to generate text
* Auto* loaders for tokenizers and models
* Model-specific loaders

Recall the `xsum` dataset from the **Summarization** section above:

In [14]:
display(xsum_sample.to_pandas())

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984
5,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,20836172
7,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467
8,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845
9,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,30358490


#### Search and sampling in inference
You may see parameters like `num_beams`, `do_sample`, etc. specified in Hugging Face pipelines.  These are inference configurations.

LLMs work by predicting (generating) the next token, then the next, and so on.  The goal is to generate a high probability sequence of tokens, which is essentially a search through the (enormous) space of potential sequences.

To do this search, LLMs use one of two main methods:
* **Search**: Given the tokens generated so far, pick the next most likely token in a "search."
  * **Greedy search** (default): Pick the single next most likely token in a greedy search.
  * **Beam search**: Greedy search can be extended via beam search, which searches down several sequence paths, via the parameter `num_beams`.
* **Sampling**: Given the tokens generated so far, pick the next token by sampling from the predicted distribution of tokens.
  * **Top-K sampling**: The parameter `top_k` modifies sampling by limiting it to the `k` most likely tokens.
  * **Top-p sampling**: The parameter `top_p` modifies sampling by limiting it to the most likely tokens up to probability mass `p`.

You can toggle between search and sampling via parameter `do_sample`.

For more background on search and sampling, see [this Hugging Face blog post](https://huggingface.co/blog/how-to-generate).

We will illustrate these various options below using our summarization pipeline.

In [15]:
# We previously called the summarization pipeline using the default inference configuration.
# This does greedy search.

summarizer(xsum_sample["document"][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [16]:
# We can instead do a beam search by specifying num_beams.
# This takes longer to run, but it might find a better (more likely) sequence of text.

summarizer(xsum_sample["document"][0], num_beams = 10)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [17]:
# Alternatively, we could use sampling.

summarizer(xsum_sample["document"][0], do_sample = True)

[{'summary_text': 'many businesses and householders were affected by flooding in Newton Stewart . the water breached a retaining wall, flooding many commercial properties . a flood alert remains in place across the Borders because of the constant rain .'}]

In [18]:
# We can modify sampling to be more greedy by limiting sampling to the top_k or top_p most likely next tokens.

summarizer(xsum_sample["document"][0], do_sample = True, top_k = 10, top_p = 0.8)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

#### Auto* loaders for tokenizers and models

We have already seen the `dataset` and `pipeline` abstractions from Hugging Face.  While a `pipeline` is a quick way to set up an LLM for a given task, the slightly lower-level abstractions `model` and `tokenizer` permit a bit more control over options.  We will show how to use those briefly, following this pattern:

* Given input articles.
* Tokenize them (converting to token indices).
* Apply the model on the tokenized data to generate summaries (represented as token indices).
* Decode the summaries into human-readable text.

We will first look at the [Auto* classes](https://huggingface.co/docs/transformers/model_doc/auto) for tokenizers and model types which can simplify loading pre-trained tokenizers and models.

API docs:
* [AutoTokenizer](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoTokenizer)
* [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM)


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pre-trained tokenizer and model

tokenizer = AutoTokenizer.from_pretrained(
    't5-small',
    cache_dir = 'data'
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    't5-small',
    cache_dir = 'data'
)

In [20]:
# For summarization, T5-small expects a prefix "summarize: ", so we prepend that to each article as a prompt.

articles = list(map(
    lambda article: "summarize: " + article,
    xsum_sample["document"]
))

display(pd.DataFrame(articles, columns=["prompts"]))

,prompts
0,summarize: The full cost of damage in Newton S...
1,summarize: A fire alarm went off at the Holida...
2,summarize: Ferrari appeared in a position to c...
3,"summarize: John Edward Bates, formerly of Spal..."
4,summarize: Patients and staff were evacuated f...
5,summarize: Simone Favaro got the crucial try w...
6,"summarize: Veronica Vanessa Chango-Alverez, 31..."
7,summarize: Belgian cyclist Demoitie died after...
8,"summarize: Gundogan, 26, told BBC Sport he ""ca..."
9,summarize: The crash happened about 07:20 GMT ...


In [21]:
# Tokenize the input

inputs = tokenizer(articles, max_length=1024, return_tensors="pt", padding=True, truncation=True)

print("input_ids:")
print(inputs['input_ids'])
print("attention_mask:")
print(inputs['attention_mask'])

input_ids:
tensor([[21603,    10,    37,  ...,     0,     0,     0],
        [21603,    10,    71,  ...,     0,     0,     0],
        [21603,    10, 21945,  ..., 18002,    21,     1],
        ...,
        [21603,    10, 21768,  ...,     0,     0,     0],
        [21603,    10,  9982,  ...,     0,     0,     0],
        [21603,    10,    37,  ...,     0,     0,     0]])
attention_mask:
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


# Generate summaries

summary_ids = model.generate(
    inputs.input_ids,
    attention_mask = inputs.attention_mask,
    num_beams = 2, min_length = 0, max_length = 40
)

print(summary_ids)

In [24]:
# Decode the generated summaries

decoded_summaries = tokenizer.batch_decode(summary, skip_special_tokens = True)
display(pd.DataFrame(decoded_summaries, columns=["decoded_summaries"]))

NameError: name 'summary' is not defined

In [ ]:
end_time = time.time()
total_time = end_time - start_time
total_time = time.strftime("%H:%M:%S", time.gmtime(total_time))
print("Script execution time:", total_time)

### Summary

We've covered some common LLM applications and seen how to get started with them quickly using pre-trained models from the Hugging Face Hub.  We've also see how to tweak some configurations.

But how did we find those models for our tasks?  In the lab, you will find new pre-trained models for tasks, using the Hugging Face Hub.  You will also explore tweaking model configurations to gain intuition about their effects.

<hr>

 &copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>

<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>